In [33]:
"""
Retrieve predicted probabilities for roof material classes from ResNet-18 models (CV)
"""

import sys, os

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsat.models.classification import resnet18
from torch.nn.functional import softmax

# Custom functions (__functions.py)
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping'
results_dir = os.path.join(maindir, 'results/resnet18/')
print(results_dir)

print("Imports successful !")

/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping/results/resnet18/
Imports successful !


In [27]:
def initialize_resnet_model(n_classes, n_bands, device, learning_rate, momentum, weight_decay, patience):
    """
    Initializes the ResNet-18 model, optimizer, scheduler, scaler, and loss criterion.
    
    Args:
        n_classes (int): Number of output classes.
        n_bands (int): Number of input image bands.
        device (torch.device): Device to run the model on (CPU/GPU).
        learning_rate (float): Learning rate for the optimizer.
        momentum (float): Momentum for SGD optimizer.
        weight_decay (float): Weight decay for SGD optimizer.
        patience (int): Patience for learning rate scheduler.

    Returns:
        model (torch.nn.Module): The initialized model.
        optimizer (torch.optim.Optimizer): The optimizer.
        lr_scheduler (torch.optim.lr_scheduler): The learning rate scheduler.
        scaler (torch.cuda.amp.GradScaler): The scaler for mixed precision.
    """
    # Initialize the Resnet-18 model
    model = resnet18(n_classes, in_channels=n_bands, pretrained=False)
    # Move the model to the specified device
    if torch.cuda.device_count() >= 1:
        model = nn.DataParallel(model)
        model.to(device)
    else:
        model = nn.DataParallel(model)
        model.to(device)
        print('Made CPU parallel')

    # Define the optimizer
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    
    # Define the learning rate scheduler
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, verbose=False, min_lr=1e-6)
    
    # Initialize the scaler for mixed precision
    scaler = torch.cuda.amp.GradScaler()

    return model, optimizer, lr_scheduler, scaler

In [11]:
# Load our image data to check on the format
stack_da_fp = os.path.join(maindir,'data/spatial/mod/dc_data/planet-data/dc_0623_psscene8b_final_norm.tif')
# stack_da_fp = os.path.join(homedir,'opp-data/dc_0623_psscene8b_final_norm.tif')
stack_da = rxr.open_rasterio(stack_da_fp, mask=True, cache=False).squeeze()
n_bands = stack_da.values.shape[:1][0] # get a list of band names
print(
    f"shape: {stack_da.rio.shape}\n"
    f"bands: {n_bands}\n"
    f"resolution: {stack_da.rio.resolution()}\n"
    f"bounds: {stack_da.rio.bounds()}\n"
    f"sum: {stack_da.sum().item()}\n"
    f"CRS: {stack_da.rio.crs}\n"
    f"NoData: {stack_da.rio.nodata}\n"
    f"Array: {stack_da}"
)
del stack_da

shape: (8223, 6714)
bands: 6
resolution: (3.0, -3.0)
bounds: (315267.0, 4294629.0, 335409.0, 4319298.0)
sum: 8.181640625
CRS: EPSG:32618
NoData: None
Array: <xarray.DataArray (band: 6, y: 8223, x: 6714)> Size: 1GB
[331255332 values with dtype=float32]
Coordinates:
  * band         (band) int64 48B 1 2 3 4 5 6
  * x            (x) float64 54kB 3.153e+05 3.153e+05 ... 3.354e+05 3.354e+05
  * y            (y) float64 66kB 4.319e+06 4.319e+06 ... 4.295e+06 4.295e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('nir', 'NDBIbg', 'NDBIrg', 'NISI', 'MNF1', 'NISI5x5')


In [ ]:
# Perform predictions on the holdout data for each model

In [8]:
# Load the holdout data
holdout_df = gpd.read_file(os.path.join(results_dir, 'cv-results/dc-resnet18_cv_holdout_ref.gpkg'))
print("Holdout set class distribution:\n", holdout_df[['class_code','code']].value_counts())

Holdout set class distribution:
 class_code  code
CS          0       7427
ME          1       7373
SL          2       3054
UR          3        256
WS          5        231
TL          4        185
SH          6        157
Name: count, dtype: int64


In [9]:
# Create dictionaries for mapping
code_mapping = dict(zip(holdout_df['code'], holdout_df['class_code']))  # Mapping to original 'class_code'
desc_mapping = dict(zip(holdout_df['code'], holdout_df['description']))
print(f'Code map: \n{code_mapping}\nDescription map: \n{desc_mapping}')

Code map: 
{0: 'CS', 1: 'ME', 2: 'SL', 3: 'UR', 4: 'TL', 5: 'WS', 6: 'SH'}
Description map: 
{0: 'Composition Shingle', 1: 'Metal', 2: 'Slate', 3: 'Urethane', 4: 'Tile', 5: 'Wood shake/shingle', 6: 'Shingle'}


In [13]:
# Best params from tuning
params = {'window_size': 78, 'batch_size': 64, 'learning_rate': 0.01, 'weight_decay': 0.01, 'momentum': 0.85, 'patience': 5}
print(f'Model params: {params}')

Model params: {'window_size': 78, 'batch_size': 64, 'learning_rate': 0.01, 'weight_decay': 0.01, 'momentum': 0.85, 'patience': 5}


In [16]:
# Load the holdout data into Torch
holdout_ds = RoofImageDatasetPlanet(holdout_df[['geometry', 'code']], img_path=stack_da_fp, n_bands=n_bands, img_dim=params['window_size'])
holdout_loader = DataLoader(holdout_ds, batch_size=params['batch_size'] * 2, num_workers=2, shuffle=False, pin_memory=True)
print("Holdout data loaded !")

Holdout data loaded !


In [18]:
# Define whether to leverage cpu or gpu (for my local machine it is only cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
print(f'Using {device} for model eval ...')

Using cpu for model eval ...


In [34]:
n_folds = 5
n_classes = len(code_mapping.keys())

# Initialize list to store the results:
all_preds = []
# loop the trained models and make predictions
for fold_idx in range(1, n_folds + 1):
    print(f"Making predictions with model from fold {fold_idx}")

    # Load the trained model for the current fold
    model_fp = os.path.join(results_dir, f'cv-models/dc-resnet18_fold{fold_idx}.pth')
    print(f"Loading model from path: {model_fp}")
    checkpoint = torch.load(model_fp, map_location=device)

    # Initialize the model architecture
    model, _, _, _ = initialize_resnet_model(
        n_classes=n_classes,
        n_bands=n_bands,
        device=device,
        learning_rate=params['learning_rate'],
        momentum=params['momentum'],
        weight_decay=params['weight_decay'],
        patience=params['patience']
    )

    # Load the trained weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()  # Set the model to evaluation mode
    
    true_codes = []
    pred_codes = []
    pred_probs = [] 
    
    # Make predictions on the holdout data
    with torch.no_grad():
        for idx, sample in enumerate(holdout_loader):
            image, label = sample['image'].to(device), sample['code'].to(device)

            # Get the model's predictions
            output = model(image.float())
            # Apply softmax to get probabilities
            probabilities = softmax(output, dim=1).cpu().numpy()
            # Apply argmax to get the predicted class
            predictions = output.argmax(dim=1).cpu().numpy()

            # Store true labels, predictions, and probabilities
            true_codes.extend(label.cpu().numpy())
            pred_codes.extend(predictions)
            pred_probs.extend(probabilities)

            if idx % 10 == 0:
                print(f"\tProcessed {idx * params['batch_size']} samples")

    # Map true and predicted labels to their class codes
    true_class_codes = [code_mapping[code] for code in true_codes]
    pred_class_codes = [code_mapping[pred] for pred in pred_codes]

    # Store true and predicted labels in a dataframe
    fold_df = pd.DataFrame({
        'fold_idx': fold_idx,         
        'true_code': true_codes,
        'pred_code': pred_codes,
        'true_label': true_class_codes,
        'pred_label': pred_class_codes
    })

    # Add predicted probabilities for each class as seperate columns
    for class_idx in range(n_classes):
        class_label = code_mapping[class_idx]
        fold_df[f"{class_label}"] = [prob[class_idx] for prob in pred_probs]
    
    all_preds.append(fold_df)

    del model
    if device == 'cuda':
        torch.cuda.empty_cache()
    gc.collect()

# concatenate across folds
preds_df = pd.concat(all_preds, ignore_index=True)

# Save the predictions for later analysis
preds_df.to_csv(os.path.join(results_dir, 'cv-results/dc-resnet18_cv_holdout_prob_preds.csv'), index=False)
print("Completed predictions for all folds!")

Making predictions with model from fold 1
Loading model from path: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping/results/resnet18/cv-models/dc-resnet18_fold1.pth
Made CPU parallel
	Processed 0 samples
	Processed 640 samples
	Processed 1280 samples
	Processed 1920 samples
	Processed 2560 samples
	Processed 3200 samples
	Processed 3840 samples
	Processed 4480 samples
	Processed 5120 samples
	Processed 5760 samples
	Processed 6400 samples
	Processed 7040 samples
	Processed 7680 samples
	Processed 8320 samples
	Processed 8960 samples


NameError: name 'class_mapping' is not defined